<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/TXT_SarcarsmDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf

## Data Gathering

In [2]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json

--2020-09-26 05:51:02--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json.1’

sarcasm.json.1      100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2020-09-26 05:51:02 (289 MB/s) - ‘sarcasm.json.1’ saved [5643545/5643545]



In [3]:
df = pd.read_json("./sarcasm.json")
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
texts = df["headline"].values
labels = df["is_sarcastic"].values

## Data Preparation

### Tokenization

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
VOCAB_SIZE = 10000
tokenizer = Tokenizer(VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

### Create samples

In [7]:
df["headline"].apply(lambda x : len(x)).describe()

count    26709.000000
mean        60.910629
std         19.184423
min          7.000000
25%         48.000000
50%         61.000000
75%         73.000000
max        254.000000
Name: headline, dtype: float64

In [8]:
MAX_SEQ_LEN = 128
BUFFER_SIZE = 1024

text_ids = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQ_LEN, padding="post")
samples = tf.data.Dataset.from_tensor_slices((text_ids, labels)).shuffle(BUFFER_SIZE)

### Create dataset

In [9]:
BATCH_SIZE = 64

test_size = 0.2
num_samples = sum(1 for x in samples)
num_train_samples = int(test_size * num_samples)
ds_train = samples.take(num_train_samples).batch(BATCH_SIZE)
ds_val = samples.skip(num_train_samples).batch(BATCH_SIZE)

## Model Architecture

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, GlobalAveragePooling1D, Dense, GlobalMaxPooling1D

In [11]:
EMBED_DIM = 32

model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBED_DIM))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])

In [13]:
history = model.fit(ds_train, epochs=50, validation_data=ds_val)

Epoch 1/50
84/84 [==============================] - 1s 15ms/step - loss: 0.6862 - acc: 0.5579 - val_loss: 0.6840 - val_acc: 0.5611
Epoch 2/50
84/84 [==============================] - 1s 12ms/step - loss: 0.6828 - acc: 0.5630 - val_loss: 0.6821 - val_acc: 0.5603
Epoch 3/50
84/84 [==============================] - 1s 13ms/step - loss: 0.6795 - acc: 0.5641 - val_loss: 0.6799 - val_acc: 0.5605
Epoch 4/50
84/84 [==============================] - 1s 13ms/step - loss: 0.6765 - acc: 0.5624 - val_loss: 0.6767 - val_acc: 0.5606
Epoch 5/50
84/84 [==============================] - 1s 14ms/step - loss: 0.6743 - acc: 0.5600 - val_loss: 0.6732 - val_acc: 0.5613
Epoch 6/50
84/84 [==============================] - 1s 14ms/step - loss: 0.6701 - acc: 0.5604 - val_loss: 0.6692 - val_acc: 0.5606
Epoch 7/50
84/84 [==============================] - 1s 15ms/step - loss: 0.6644 - acc: 0.5643 - val_loss: 0.6642 - val_acc: 0.5614
Epoch 8/50
84/84 [==============================] - 1s 14ms/step - loss: 0.6596 - a

In [14]:
model.evaluate(ds_val)

334/334 [==============================] - 1s 2ms/step - loss: 0.3903 - acc: 0.8285


[0.3902556300163269, 0.828528642654419]